In [1]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import torch

torch.cuda.is_available()

True

In [ ]:
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader

train, test = dataset_loader((128, 128))

# todo change name of KFoldController and also check it is shuffled
# I am using the KFoldController class to simply get 20% of the dataset as a validation fold. I won't do any
# k-Fold CV on the parameters search or else this will take forever
dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)

In [ ]:
import keras
from models.naive_dnn_gen.naive_dnn import NaiveDnnContainer
import keras_tuner

hyperparameters = keras_tuner.HyperParameters()

hypermodel = NaiveDnnContainer((3, 128, 128))
tuner = keras_tuner.Hyperband(
    hypermodel,
    objective='val_loss',
    directory="naive_dnn_search",
    executions_per_trial=10,  # should be highest possible
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=100, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
             ])

In [ ]:
models = tuner.get_best_models(num_models=18)
best_model = models[0]
best_model.summary()

# todo: Forse per il momento mi conviene fissare lr e altgri parametri di learning per poi riprenderli

# Might Grayscale images contain enough information to correctly find the solutions of our task?


In [4]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
from models.naive_dnn_gen.naive_dnn import NaiveDnnContainer
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader
## Let's try Grayscale
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((128, 128), is_grayscale=True)

# todo change name of KFoldController and also check it is shuffled
# I am using the KFoldController class to simply get 20% of the dataset as a validation fold. I won't do any
# k-Fold CV on the parameters search or else this will take forever
dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)
hypermodel = NaiveDnnContainer((1, 128, 128))
# We are learning the structure of the network with fixed parameters for learning.
# As we know hyperparamters of NN are: Neurons, Layers, Optimizer, Learning-Rate, Epochs, Batch size etc
hyperparameters.Fixed("lr", value=1e-3)
hyperparameters.Fixed("dropout_0", value=False)
hyperparameters.Fixed("dropout_1", value=False)
hyperparameters.Fixed("dropout_2", value=False)
hyperparameters.Fixed("dropout_3", value=False)

tuner = keras_tuner.Hyperband(
    hypermodel,
    hyperparameters=hyperparameters,
    hyperband_iterations=3,
    objective='val_loss',
    tune_new_entries=True,
    overwrite=True,
    directory="naive_dnn_search",
    project_name="grayscale_fixed_lr" # should be highest possible
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=100, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
             ])

Trial 132 Complete [00h 04m 40s]
val_loss: 0.6651219129562378

Best val_loss So Far: 0.6651219129562378
Total elapsed time: 02h 36m 43s

Search: Running Trial #133

Value             |Best Value So Far |Hyperparameter
0.001             |0.001             |lr
False             |False             |dropout_0
False             |False             |dropout_1
False             |False             |dropout_2
False             |False             |dropout_3
2                 |2                 |layers
64                |128               |layer_0
704               |1408              |layer_1
12                |12                |tuner/epochs
4                 |4                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
2                 |2                 |tuner/round
0100              |0098              |tuner/trial_id
Epoch 5/12
 88/119 ━━━━━━━━━━━━━━━━━━━━ 7s 246ms/step - accuracy: 0.5715 - loss: 0.6795

KeyboardInterrupt: 